## Better Way 31
### 인자에 대해 이터레이션 할 때는 방어적이 돼라

<br>

### 객체가 원소로 들어간 리스트를 함수가 파라미터로 받았을 때, 이 리스트를 반복 이터레이션 하는 경우 
Ex) 미국텍사스 주의 여행자 수 분석, 데이터 집합은 도시별 방문자 수. 이때 각 도시가 전체 여행자 수중에서 차지하는 비율 계산 


####  각 도시의 방문자 수 / 입력전체 합계 구하는 정규화 함수 

In [1]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

#### 위 함수는 방문제 데이터가 들어있는 리스트를 입력으로 넣으면 잘 작동

In [2]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


#### 이 코드의 규모확장성을 높이려면 제너레이터를 정의하자.

In [10]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [26]:
it = read_visits('my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


#### 위 코드 실행시 빈 리스트가 출력된다
이터레이터가 결과를 단 한 번만 만들어내기 때문에 아래의 경우엔 아무 결과도 얻을 수 없다.
* 이미 StopIteration 예외가 발생한 이터레이터
* 제너레이터를 다시 이터레이션 했을 때


In [28]:
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it)) # 이미 모든 원소를 다 소진했다.

[15, 35, 80]
[]


#### 이미 소진터레이터에 대해 이터레이션을 수행해도 오류발생하지 않음
for 루프, 리스트 생성자, 그 외 파이썬 표준 라이브러리의 많은 함수가 일반적인 연산 도중에 **StopIteration 예외 발생을 가정**         
-> 출력이 없는 이터레이터와 이미 소진된 이터레이터를 구분할 수 없다.
#### 이터레이터를 명시적으로 소진시키고 그 전체내용을 리스트에 넣어서 해결
그후 원하는 수만큼 이터레이션 수행 가능

### 입력 이터레이터를 방어적으로 복사하는 코드

In [29]:
def normalize_copy(numbers):
    numbers_copy = list(numbers)   # 이터레이터 복사
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

#### 위 함수는 read_visits 제너레이터가 반환하는 값에도 잘 작동

In [30]:
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### 위 접근법의 문제점
* 입력 이터레이터의 내용을 복사하여 메모리를 엄청나게 많이 사용할 수 있다.
* 메모리 부족으로 인한 프로그램 중단 가능성
* 규모확장성의 문제 발생   
  -> 처음에 read_visits를 바꿔 쓰기로 결정한 근본적인 이유
  
### 다른 해결법: 호출될때 마다 새로운 이터레이터를 반환하는 함수를 받기  

In [31]:
def normalize_func(get_iter):   # 새 이터레이터
    total = sum(get_iter())
    result = []
    for value in get_iter():   # 새 이터레이터
        percent = 100 * value / total
        result.append(percent)
    return result

In [32]:
path = 'my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path)) # normalize_func사용시 매번 제너레이터 호출해 
print(percentages)                                       # 새 이터레이터 만드는 lambda식 전달
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


### 작동하지만 lambda함수를 넘기는 건 보기 좋지 않다.

<br>

### 더 나은 해결법: 이터레이터 프로토콜(iterator protocol)을 구현한 새로운 컨테이너 클래스 제공

#### 이터레이터 프로토콜
파이썬의 for루프나 그와 연관된 식들이 컨테이너 타입의 내용을 방문할 때 사용하는 절차. 

1. 파이썬에서 ```for x in foo``` 사용하면 실제로는 ```iter(foo)```를 호출.
2. ```iter```내장함수는 ```foo.__iter__```특별메서드를 호출.
3. ```__iter__```는 반드시 이터레이터 객체(```__next__```객체를 정의한)를 반환해야만 한다 
4. ```for```루프는 반환받은 이터레이터 객체가 데이터를 소진(StopIteration예외발생)할 때까지 반복적으로 해당 객체에 대한 next내장함수 호출

#### 코드를 작성할 때 정의하는 클래스에서 __iter__메서드를 제너레이터로 구현하기만 하면 위의 모든 동작을 만족
### 이터러블 컨테이너 클래스 정의 코드

In [34]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

#### 원래의 normalize함수에 새로운 컨테이너를 넘기면 변경없이도 잘 작동

In [35]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


#### 작동이유
1. normalize 함수안의 sum메서드가 ReadVisits.\__iter\__ 를 호출하여 새로운 이터레이터 객체를 할당함
2. 각숫자를 정규화하기 위한 for루프도 \__iter\__를 호출해 두번째 이터레이터 객체 생성
3. 1.과 2.의 이터레이터는 서로 독립적으로 진행 및 소진됨   
   -> 모든 이터레이터 값을 볼 수 있는 별도의 이터레이션 생성 
   
#### 입력데이터를 여러번 읽는 다는 것이 유일한 단점   

<br>

### 파라미터값이 단순 이터레이터가 아니어도 동작하는 함수 정의

* iter(이터레이터A) -> 이터레이터A 그대로 반환 
* iter(컨테이너B) -> 매번 새로운 이터레이터 객체 B,C,D,E... 반환

#### 위 두 경우중 어느 쪽인지 검사해 반복적으로 이터레이션 불가능할 때는 TypeError 발생시켜서 거부

In [37]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers:   # 이터레이터 -- 나쁨!
        raise TypeError('컨테이너를 제공해야 합니다.')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

<class 'list_iterator'> <class 'list'>
2307996030288 2307996081984
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [40]:
dir([1, 2, 3].__iter__())

In [42]:
x = [1, 2, 3].__iter__()
dir(x)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__length_hint__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__length_hint__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']